In [59]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from warnings import filterwarnings


In [60]:
diabetes = pd.read_csv("diabetes.csv")
df = diabetes.copy()
df = df.dropna()
y = df["Outcome"]
x = df.drop(["Outcome"],axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=42)

In [61]:
from sklearn.preprocessing import StandardScaler

In [62]:
scaler = StandardScaler()

In [63]:
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [64]:
from sklearn.neural_network import MLPClassifier

In [65]:
mlpc = MLPClassifier().fit(x_train_scaled,y_train)

C:\Users\Emrah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [66]:
y_pred = mlpc.predict(x_test_scaled)
accuracy_score(y_pred,y_test)

0.7316017316017316

# Model Tuning

In [67]:
?mlpc

Type:        MLPClassifier
String form: MLPClassifier()
File:        c:\users\emrah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py
Docstring:  
Multi-layer Perceptron classifier.

This model optimizes the log-loss function using LBFGS or stochastic
gradient descent.

.. versionadded:: 0.18

Parameters
----------
hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
    The ith element represents the number of neurons in the ith
    hidden layer.

activation : {'identity', 'logistic', 'tanh', 'relu'}, default='relu'
    Activation function for the hidden layer.

    - 'identity', no-op activation, useful to implement linear bottleneck,
      returns f(x) = x

    - 'logistic', the logistic sigmoid function,
      returns f(x) = 1 / (1 + exp(-x)).

    - 'tanh', the hyperbolic tan function,
      returns f(x) = tanh(x).

    - 'relu', the rectified linear unit function,
      returns f(x) = max(0, x)

solver : {'lbfgs', 'sgd', 'adam'}, default='a

In [68]:
mlpc_params = {"alpha": [0.1,0.01,0.02,0.005,0.0001,0.00001],
               "hidden_layer_sizes":[(10,10,10),
                                    (50,50,50),
                                    (50,50),
                                    (3,5),
                                    (5,3)],
               "solver" : ["lbfgs","adam","sgd"],
               "activation":["relu","logistic"]}

In [69]:
mlpc = MLPClassifier()
mlpc_cv_model = GridSearchCV(mlpc,mlpc_params,
                            cv=10,
                            n_jobs=-1,
                            verbose=2)
mlpc_cv_model.fit(x_train_scaled,y_train)



Fitting 10 folds for each of 180 candidates, totalling 1800 fits


C:\Users\Emrah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=10, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005, 0.0001, 1e-05],
                         'hidden_layer_sizes': [(10, 10, 10), (50, 50, 50),
                                                (50, 50), (3, 5), (5, 3)],
                         'solver': ['lbfgs', 'adam', 'sgd']},
             verbose=2)

In [58]:
print("En iyi parametreler: " + str(mlpc_cv_model.best_params_))

En iyi parametreler: {'activation': 'relu', 'alpha': 1e-05, 'hidden_layer_sizes': (50, 50, 50), 'solver': 'sgd'}


In [70]:
mlpc_tuned = MLPClassifier(activation="logistic",
                           alpha= 1e-05,
                           hidden_layer_sizes = (50, 50, 50),
                           solver="sgd")

In [71]:
mlpc_tuned.fit(x_train_scaled,y_train)

MLPClassifier(activation='logistic', alpha=1e-05,
              hidden_layer_sizes=(50, 50, 50), solver='sgd')

In [72]:
y_pred = mlpc_tuned.predict(x_test_scaled)
accuracy_score(y_test,y_pred)

0.6536796536796536